In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

import warnings
from copy import deepcopy
import gc
from collections import Counter

%matplotlib inline

In [2]:
def score_model(model, X, y, threshold=0.5):  # 0.41 is the best for now
    pred = model.predict_proba(X)[:, 1]
    roc_auc = roc_auc_score(y, pred, multi_class='ovo')
    return roc_auc

In [3]:
%cd ../

/home/chervovn04/Programming/hackathons/2022/digital_breakout_885303


In [4]:
cat_features = ["type", "is_privatecategory", "class", "is_in_yandex", "mailctg", "directctg", "mailtype"]

def to_cat_features(df):
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].apply(lambda text: str(text))
        
X_train = pd.read_csv('data/X_train_prepared.csv')
y_train = pd.read_csv('data/y_train_prepared.csv')
X_val = pd.read_csv('data/X_val_prepared.csv')
y_val = pd.read_csv('data/y_val_prepared.csv')
test_df = pd.read_csv('data/test_df_prepared.csv')
test_ids = pd.read_csv('data/test_ids.csv')

to_cat_features(X_train)
to_cat_features(X_val)
to_cat_features(test_df)

/tmp/ipykernel_50657/1192962467.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  X_train = pd.read_csv('data/X_train_prepared.csv')
/tmp/ipykernel_50657/1192962467.py:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  X_val = pd.read_csv('data/X_val_prepared.csv')
/tmp/ipykernel_50657/1192962467.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('data/test_df_prepared.csv')


In [5]:
X = pd.concat([X_train, X_val, test_df], ignore_index=True) 
y = pd.concat([pd.Series(np.zeros(X_train.shape[0] + X_val.shape[0])), pd.Series(np.ones(test_df.shape[0]))], ignore_index=True)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=0)

In [7]:
model = CatBoostClassifier(random_state=0, max_depth=8, verbose=10, iterations=30, auto_class_weights="Balanced", cat_features=cat_features)

In [8]:
model.fit(X_train, y_train) 

/home/chervovn04/Programming/hackathons/2022/digital_breakout_885303/venv/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


Learning rate set to 0.5
0:	learn: 0.6931273	total: 675ms	remaining: 19.6s
10:	learn: 0.6929099	total: 7.17s	remaining: 12.4s
20:	learn: 0.6927319	total: 13.5s	remaining: 5.78s
29:	learn: 0.6926014	total: 19.3s	remaining: 0us


In [9]:
score_model(model, X_test, y_test)

0.5000389711315667

In [10]:
sorted(list(zip(X_train.columns, model.feature_importances_)), key = lambda x: -x[1])

[('weight', 9.135518038520688),
 ('weight_mfi', 8.697560129957775),
 ('index_oper', 7.168197343984904),
 ('mailtype', 6.833989710897045),
 ('price_mfi', 6.706454455882057),
 ('type', 6.503267939681439),
 ('total_qty_over_index_and_type', 6.302740255535803),
 ('mailctg', 6.24260833292949),
 ('dist_qty_oper_login_1', 6.014780104721615),
 ('total_qty_oper_login_1', 4.584129883621805),
 ('transport_pay', 4.451922445321322),
 ('priority', 4.285536296207506),
 ('total_qty_over_index', 4.03113146744381),
 ('class', 3.8106469628746265),
 ('total_qty_oper_login_0', 3.799356052786485),
 ('is_wrong_phone_number', 3.1279437288502367),
 ('is_return', 1.7371254846138235),
 ('goods_count', 1.379770707226256),
 ('is_wrong_address', 1.1529467300690266),
 ('is_wrong_rcpn_name', 1.0689995610504153),
 ('postmark', 1.0032255772819028),
 ('is_102102_operator', 0.4835093866987517),
 ('is_620984_operator', 0.4467644103510296),
 ('is_privatecategory', 0.31225116639565925),
 ('is_wrong_sndr_name', 0.24169500296